In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [2]:
def clean_hubspot_export(csv = 'all-companies.csv'):
    all_companies_info_df = pd.read_csv(csv)
    all_companies_info_df = all_companies_info_df[all_companies_info_df['ICP?'] != 'Non-ICP']
    all_companies_info_df = all_companies_info_df.reset_index().drop('index', axis=1)
    all_companies_df = all_companies_info_df.reindex(['Record ID', 'Company name', 'Website URL', 'City', 'State', 'Full Address', 'Record Type', 'ICP?', 'Phone Number'], axis=1)
    all_companies_df = all_companies_df.rename({'Website URL':'Website'}, axis=1)
    all_companies_df = all_companies_df.fillna('')

    return all_companies_df

In [3]:
def clean_data(_df):
    for i in range(len(_df)):
        _df['Phone Number'] = _df['Phone Number'].astype(str)
        
        if ',' in _df['Website'][i]:
            _df['Website'][i] = _df['Website'][i].split(',')[0]
        _df['Website'][i] = _df['Website'][i].replace('https://', '').replace('http://', '').replace('www.', '').rstrip('/')

        num=''
        for j in range(len(_df['Phone Number'][i])):
            char = _df['Phone Number'][i][j]
            if char.isdigit():
                num += char
        num = num.lstrip('1')
        _df['Phone Number'][i] = num
    return _df

In [5]:
def match(df_1, df_2):
    # df_1 = external Data
    # df_2 = hubspot Data

    matched_df = pd.DataFrame()
    df_1['Hubspot ID'] = ''
    df_1['Hubspot Name'] = ''
    df_1['Matched On'] = ''

    for i in range(len(df_1)): 
        for j in range(len(df_2)): 
            # Assign the city as the city in the name for Hubspot DF
            try:
                df_2['City'][j] = df_2['Company name'][j].split(' - ')[1]
            except:
                df_2['City'][j] = df_2['City'][j]

            ### Matching process
            # Match on website
            if (df_1['Website'][i] != '' and df_2['Website'][j] != '' and df_1['Website'][i] == df_2['Website'][j]) and (df_2['City'][j] in df_1['Full Address'][i] and df_2['City'][j] != ''):
                df_1['Hubspot ID'][i] = df_2['Record ID'][j]
                df_1['Hubspot Name'][i] = df_2['Company name'][j]
                df_1['Matched On'][i] = 'Website'

            # Match on Phone Number
            elif (df_1['Phone Number'][i] == df_2['Phone Number'][j]) and (df_2['Phone Number'][j] != '') and (df_2['City'][j] in df_1['Full Address'][i] and df_2['City'][j] != ''):
                df_1['Hubspot ID'][i] = df_2['Record ID'][j]
                df_1['Hubspot Name'][i] = df_2['Company name'][j]
                df_1['Matched On'][i] = 'Phone'


            # Match on Name
            elif (df_1['Company name'][i] in df_2['Company name'][j] or df_2['Company name'][j] in df_1['Company name'][i]) and (df_2['City'][j] in df_1['Full Address'][i] and df_2['City'][j] != ''):
                df_1['Hubspot ID'][i] = df_2['Record ID'][j]
                df_1['Hubspot Name'][i] = df_2['Company name'][j]
                df_1['Matched On'][i] = 'Name'
            else:
                continue
    
    return df_1

### EMS Schools

In [2]:
ems_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1mUbB48_bVclseWFWOnMyfsrIKhPGUP3VmpO6JZZPFu8/edit#gid=14726155628')
ems_school_sheet = ems_wb.worksheet('Project 3 - Data')

In [35]:
ems_schools_df = pd.DataFrame(ems_school_sheet.get_all_records())
ems_schools_df = ems_schools_df[ems_schools_df['ICP?'] == 'ICP']
ems_schools_df = ems_schools_df.reset_index().drop('index', axis=1)
# ems_schools_df = ems_schools_df.reindex(['Company name', 'Website', 'City', 'State', 'Full Address', 'Phone Number'], axis=1)
ems_schools_df = ems_schools_df.fillna('')

all_companies_info_df = pd.read_csv('all-companies 5.csv')
all_companies_info_df = all_companies_info_df[all_companies_info_df['ICP?'] != 'Non-ICP']
all_companies_info_df = all_companies_info_df.reset_index().drop('index', axis=1)
all_companies_df = all_companies_info_df.reindex(['Company ID', 'Company name', 'Website URL', 'City', 'State', 'Full Address', 'Record Type', 'ICP?', 'Phone Number'], axis=1)
all_companies_df = all_companies_df.rename({'Website URL':'Website'}, axis=1)
all_companies_df = all_companies_df.fillna('')

In [41]:
ems_schools_df = clean_data(ems_schools_df)
all_companies_df = clean_data(all_companies_df)

matched_df = match(ems_schools_df, all_companies_df)

# matched_sheet = ems_wb.worksheet('Matched 2')
# set_with_dataframe(matched_sheet, matched_df)

9/8/2022 

### Accreditor Pull

In [6]:
all_companies_df = clean_hubspot_export()
all_companies_df = clean_data(all_companies_df)

/var/folders/k6/gc55jlbs00z7sxrx8y36_2940000gn/T/ipykernel_31503/4069183239.py:2: DtypeWarning: Columns (17,22,23,45,63,83,84,113,116,119,121,149,161,165,175,189) have mixed types. Specify dtype option on import or set low_memory=False.
  all_companies_info_df = pd.read_csv(csv)


In [15]:
accred_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1S32UvSdTLSd_7jRf3fP_yt-ahhgPUm31QiP8BYp-p1o/edit#gid=0')
# accsc_df = pd.DataFrame(accred_wb.worksheet('ACCSC').get_all_records())
# abhes_df = pd.DataFrame(accred_wb.worksheet('ABHES').get_all_records())
# accet_df = pd.DataFrame(accred_wb.worksheet('ACCET').get_all_records())
# coe_df = pd.DataFrame(accred_wb.worksheet('COE').get_all_records())
# acics_df = pd.DataFrame(accred_wb.worksheet('ACICS').get_all_records())
deac_df = pd.DataFrame(accred_wb.worksheet('DEAC').get_all_records())

In [16]:
# accsc_df = clean_data(accsc_df)
# abhes_df = clean_data(abhes_df)
# accet_df = clean_data(accet_df)
# coe_df = clean_data(coe_df)
# acics_df = clean_data(acics_df)
deac_df = clean_data(deac_df)

In [17]:
# abhes_matched_df = match(abhes_df, all_companies_df)
# accet_matched_df = match(accet_df, all_companies_df)
# coe_matched_df = match(coe_df, all_companies_df)
# acics_matched_df = match(acics_df, all_companies_df)
deac_matched_df = match(deac_df, all_companies_df)

In [13]:
# abhes_matched_df.to_csv('abhes_matched.csv')
# accet_matched_df.to_csv('accet_matched.csv')
# coe_matched_df.to_csv('coe_matched.csv')
# acics_matched_df.to_csv('acics_matched.csv')
deac_matched_df.to_csv('deac_matched.csv')